#### Select negative examples 

1. Find candidate not-phosphosites (unsite) from excluding identified phosphorylations. 
2. Filter out candidates that do not appear on peptides with sufficient evidence. 
3. Get structural annotation of all candidates. 
4. Get structural annotation of positive examples.
5. Select from negative candidates such that same amount in distribution (i.e. histogram bins)

### Select negative examples

Use structural annotations to select negative examples that follow the same distribution as positive examples.

In [3]:
# Autoreload 
%load_ext autoreload
%autoreload 2
import pandas as pd 
import numpy as np

from pathlib import Path
from typing import Union, List, Tuple, Dict, Optional

# STFU
pd.options.mode.chained_assignment = None  # default='warn'
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#uniprot_ids = 

In [8]:
# home directory 
cif_dir = Path.home() / "STRUCTURAL_MOTIFS" / "DATA" / "AF_HUMAN_CIF"
cif_dir.is_dir()


# TODO: add symlink to cif_dir in this directory

pae_dir = Path.home() / "STRUCTURAL_MOTIFS" / "DATA" / "AF_HUMAN_PAE"

True

In [1]:
import structuremap.utils
structuremap.utils.set_logger()
from structuremap.processing import download_alphafold_cif, download_alphafold_pae, format_alphafold_data, annotate_accessibility, get_smooth_score, annotate_proteins_with_idr_pattern, get_extended_flexible_pattern, get_proximity_pvals, perform_enrichment_analysis, perform_enrichment_analysis_per_protein, evaluate_ptm_colocalization, extract_motifs_in_proteome
from structuremap.plotting import plot_enrichment, plot_ptm_colocalization

verbose = True
valid_proteins_cif, invalid_proteins_cif, existing_proteins_cif = download_alphafold_cif(
    proteins=uniprot_ids,
    out_folder=cif_dir)


if verbose: print("Downloading PAE into {self.pae_dir}...")
valid_proteins_pae, invalid_proteins_pae, existing_proteins_pae = download_alphafold_pae(
    proteins=uniprot_ids,
    out_folder=pae_dir, 
)

"""Annotate AlphaFold data"""
alphafold_annotation = format_alphafold_data(
    directory=cif_dir, 
    protein_ids=uniprot_ids)

"""Annotate pPSE"""
full_sphere_exposure = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=24, 
    max_angle=180, 
    error_dir=self.pae_dir)

alphafold_accessibility = alphafold_annotation.merge(
    full_sphere_exposure, how='left', on=['protein_id','AA','position'])

part_sphere_exposure = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=12, 
    max_angle=70, 
    error_dir=self.pae_dir)

alphafold_accessibility = alphafold_accessibility.merge(
    part_sphere_exposure, how='left', on=['protein_id','AA','position'])

alphafold_accessibility['high_acc_5'] = np.where(alphafold_accessibility.nAA_12_70_pae <= 5, 1, 0)
alphafold_accessibility['low_acc_5'] = np.where(alphafold_accessibility.nAA_12_70_pae > 5, 1, 0)

"""Annotate IDRs"""
alphafold_accessibility_smooth = get_smooth_score(
    alphafold_accessibility, 
    np.array(['nAA_24_180_pae']), 
    [10])
alphafold_accessibility_smooth['IDR'] = np.where(
alphafold_accessibility_smooth['nAA_24_180_pae_smooth10']<=34.27, 1, 0)

"""Annotate short IDRs"""
alphafold_accessibility_smooth_pattern = annotate_proteins_with_idr_pattern(
    alphafold_accessibility_smooth,
    min_structured_length = 80, 
    max_unstructured_length = 20)

alphafold_accessibility_smooth_pattern_ext = get_extended_flexible_pattern(
    alphafold_accessibility_smooth_pattern, 
    ['flexible_pattern'], [5])

2023-10-05 18:01:18> NumExpr defaulting to 8 threads.


NameError: name 'uniprot_ids' is not defined